In [0]:
from music21 import converter,instrument,note,chord
import numpy as np
import pickle

In [0]:
with open("Notes" , 'rb') as f:
    notes = pickle.load(f)

In [4]:
print("Total Elements : ",len(notes))
print("Unique Elements : ",len(set(notes)))

Total Elements :  117989
Unique Elements :  652


In [5]:
n_vocab = set(notes)
print(len(n_vocab))

652


# Preparing Sequential Data for LSTM

In [0]:
pitchnames = sorted(set(notes))

In [0]:
element_to_label = dict((ele,num) for num,ele in enumerate(pitchnames))

In [8]:
print(element_to_label['11+1+2+6'])

173


In [0]:
# No of elements used in Markov Chains to predict the next element
sequence_length = 50

In [0]:
network_input = []
network_output = []

for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length]
    seq_out = notes[i+sequence_length]
    
    network_input.append([element_to_label[ch] for ch in seq_in])
    network_output.append(element_to_label[seq_out])

In [11]:
print(len(network_input))
print(len(network_output))

117939
117939


In [12]:
print(network_input[0])

[642, 614, 152, 649, 641, 640, 584, 613, 613, 613, 613, 613, 607, 189, 585, 613, 436, 584, 629, 633, 595, 628, 596, 627, 643, 623, 626, 584, 626, 595, 584, 622, 583, 363, 32, 596, 579, 612, 290, 579, 612, 628, 615, 597, 624, 247, 612, 583, 588, 582]


In [0]:
Xtrain = np.reshape(network_input,(-1,sequence_length,1))

## Normalise training data

In [0]:
Xtrain = Xtrain/float(len(set(notes)))

In [0]:
#%tensorflow_version 1.x

In [16]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
Ytrain = to_categorical(network_output)

In [18]:
print(Xtrain.shape)
print(Ytrain.shape)

(117939, 50, 1)
(117939, 652)


# LSTM Model

In [0]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [0]:
model = Sequential()
model.add(LSTM(512,input_shape=(Xtrain.shape[1],1),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dense(512))
model.add(Dropout(0.2))
model.add(Dense(len(set(notes)),activation='softmax'))

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 512)           1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)              

In [0]:
adam = Adam(lr=1e-3)
model.compile(loss='categorical_crossentropy',optimizer=adam)

In [23]:
checkpoint = ModelCheckpoint("piano_model.hdf5",monitor='loss',save_best_only=True)

hist = model.fit(Xtrain,Ytrain,batch_size=1024,epochs=50,callbacks=[checkpoint])

Epoch 1/50
117939/117939 [==============================] - 58s 492us/step - loss: 4.6427
Epoch 2/50
117939/117939 [==============================] - 56s 477us/step - loss: 4.6132
Epoch 3/50
117939/117939 [==============================] - 56s 474us/step - loss: 4.6010
Epoch 4/50
117939/117939 [==============================] - 56s 474us/step - loss: 4.5979
Epoch 5/50
117939/117939 [==============================] - 56s 475us/step - loss: 4.5951
Epoch 6/50
117939/117939 [==============================] - 56s 476us/step - loss: 4.5937
Epoch 7/50
117939/117939 [==============================] - 56s 472us/step - loss: 4.5927
Epoch 8/50
117939/117939 [==============================] - 55s 468us/step - loss: 4.5898
Epoch 9/50
117939/117939 [==============================] - 55s 470us/step - loss: 4.5866
Epoch 10/50
117939/117939 [==============================] - 55s 470us/step - loss: 4.5848
Epoch 11/50
117939/117939 [==============================] - 55s 469us/step - loss: 4.5817
Epoch 12